# Web Scrapping BWF Historical World Rankings

Building on the from the previous project where I analysed Loh Kean Yew's match performance, I thought it will be interesting to add on ranking as a metric. We could possibly discover some trend when he plays against higher or lower ranked opponents.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

The historical rankings can be found on the BWF website at the following [link](https://corporate.bwfbadminton.com/players/historical-rankings/).

In [2]:
base_url = 'https://corporate.bwfbadminton.com/players/historical-rankings/?ryear='
endpoint = [str(i) for i in range(2012, 2023)]

In [3]:
links = []
name = []

for year in endpoint:
    count = 0
    url = base_url + year
    response = requests.get(url)
    content = response.text
    parser = BeautifulSoup(content, 'html.parser')
    body = parser.find_all('body')
    ranking_container = body[0].find_all('div', class_ = 'ranking-files')
    link_container = ranking_container[0].find_all('a')
    
    for rank in link_container:
        links.append(rank.get('href'))
        name.append(rank.text.strip() + '.xlsx')
        count += 1
    print(year,':',count)

2012 : 52
2013 : 52
2014 : 52
2015 : 53
2016 : 52
2017 : 52
2018 : 52
2019 : 53
2020 : 11
2021 : 48
2022 : 19


In [4]:
final_rank = list(zip(name, links))

In [5]:
final_rank_df = pd.DataFrame(final_rank, columns = ['rank_week_filename', 'url_link'])

In [6]:
# correcting one of the error in filename WR 2101-06-01 (Week-22) to WR 2021-06-01 (Week-22)

final_rank_df.replace('WR 2101-06-01 (Week-22).xlsx', 'WR 2021-06-01 (Week-22).xlsx', inplace = True)
final_rank = [tuple(r) for r in final_rank_df.to_numpy()]

In [7]:
final_rank_df.to_csv('filename_url.csv', index = False)

In [8]:
for i in final_rank:
    response = requests.get(i[1])
    output = open(i[0], 'wb')
    output.write(response.content)
    output.close()